In [6]:
import pandas as pd
import numpy as np

#Fase 1: Inner join, eliminación de duplicados y eliminación de Nulos con la clase Datacollect
class Datacollect:
        #se declaran los dataframes iniciales
    def __init__(self, colecciona=pd.DataFrame, coleccionb=pd.DataFrame, union=pd.DataFrame, conteo=int):
        #Se declaran las convenciones y se cargan dataframes
        self.colecciona = pd.read_excel(r'C:\Users\Ricardo Roa\Documents\rips2019\DATA\DATOS\USX.xlsx',
                                      usecols=(['IDENTIFICACION','PRIMER_APELLIDO','SEGUNDO_APELLIDO','PRIMER_NOMBRE',
                                                'SEGUNDO_NOMBRE','EDAD','SEXO']))
        self.coleccionb = pd.read_excel(r'D:\DOCUMENTOS\RROA DOCUMENTOS\PYTHON\CENSO27062019.xlsx', 
                                    usecols=(['TIPO_DOCUMENTO','IDENTIFICACION','PRIMER_APELLIDO','SEGUNDO_APELLIDO',
                                              'PRIMER_NOMBRE','SEGUNDO_NOMBRE','EDAD','SEXO']))
        self.union= union
        self.conteo = self.coleccionb.EDAD.count()
        
    def lee_dataframe(self):
        
        datos_usuario = self.colecciona
        datos_censo = self.coleccionb 
        return self.colecciona, self.coleccionb
    
    def join(self):
        #Se realiza el inner join con ambos DataFrames
        usuario=self.colecciona
        censo=self.coleccionb
        union = self.union
        union=pd.merge(left=usuario,right=censo,how="inner", left_on="IDENTIFICACION",right_on="IDENTIFICACION",suffixes=('_usuario', '_censo'),)
        self.union = union
        
    def elimina_duplicados_nulos(self):
        #Se eliminan los registros duplicados
        union = self.union
        union=union.drop_duplicates(subset=["IDENTIFICACION"], keep="first").dropna(subset=["IDENTIFICACION"])
        self.union = union
        return union
    
    
#Fase 2: Exploracion y actualizacion de los datos, eliminacion de datos duplicados y actualizacion del archivo del Censo con la clase Limpiar
class Limpiar(Datacollect):
        

    def limpiardatos(self):
        #exploracion de nulos en el dataframe del censo e inner join entre censo y usuarios.
        usuario = self.colecciona
        censo = self.coleccionb
        censo = self.coleccionb[self.coleccionb.IDENTIFICACION.isna()]
        Nuevos_usuarios=pd.merge(left=censo,right=usuario, how="inner",  
                                 left_on=['PRIMER_APELLIDO','SEGUNDO_APELLIDO','PRIMER_NOMBRE','SEGUNDO_NOMBRE'], 
                                 right_on=['PRIMER_APELLIDO','SEGUNDO_APELLIDO','PRIMER_NOMBRE','SEGUNDO_NOMBRE'],
                                 suffixes=('_datos', '_usuario_y'),)

        Data_limpia = Nuevos_usuarios.drop_duplicates(subset=["PRIMER_NOMBRE"], keep="first")
        Nueva_Data_Limpia = Data_limpia.copy()
        Nueva_Data_Limpia['IDENTIFICACION_datos'] = np.where(Nueva_Data_Limpia['IDENTIFICACION_usuario_y'] !=0, 
                                                      Nueva_Data_Limpia['IDENTIFICACION_usuario_y'], 
                                                      Nueva_Data_Limpia['IDENTIFICACION_usuario_y'])

        self.colecciona = Nueva_Data_Limpia
        return Nueva_Data_Limpia.head()    

#Fase 3 : Concatenacion de los resultados de la Fase 1 y Fase 2 utilizando la clase Concatenacion
class Concatenacion(Limpiar):
    
    def concatena(self):
        #Filtramos y concatenamos la data del censo y del usuario (Datos limpios en un nuevo DataFrame)
        Datos_censo = self.union
        Datos_Actualizados = self.colecciona
        for col in Datos_censo.keys():
            censo_columnas = Datos_censo.filter(like="censo", axis=1) 
            censo_tipoid =   Datos_censo.filter(like= "TIPO_DOCUMENTO", axis=1)
            censoid_columna = Datos_censo.filter(like="IDENTIFICACION", axis=1) 

        #insertamos la columna id y tipoid en el Dataframe
        censo_columnas.insert(0,'IDENTIFICACION',censoid_columna)
        censo_columnas.insert(1,'TIPO_DOCUMENTO',censo_tipoid)

        #Concatenamos y filtramos el Dataframe con la actualizacion de los nulos encontrados
        Datos_Actualizados = self.colecciona
        col_lis=[ ]
        contador = 0
        try:
            #insertamos los datos limpios en el dataframe (hacemos el Datawrangling de los dataframes iniciales es decir del censo y usuarios)
            for columna,item in Datos_Actualizados.iteritems():
                colx = pd.Series(columna).str.contains("_datos") | pd.Series(columna).str.contains("PRIMER") | pd.Series(columna).str.contains("SEGUNDO")
                if colx.bool() == True:
                    y = columna
                    col_lis.insert(contador,y)
                    contador = contador + 1
                    for cola in Datos_Actualizados.columns:
                        usuarios_finales = Datos_Actualizados[col_lis]
        except:
            print("Error, no se pudo realizar la operacion")

        #Renombramos las columnas del DataFrame usuario        
        Datos_finales_usuarios = usuarios_finales.rename(columns={'IDENTIFICACION_datos':'IDENTIFICACION','EDAD_datos':'EDAD',
                                                            'SEXO_datos':'SEXO'})
        #Renombramos las columnas del DataFrame censo 
        Datos_censo_final=censo_columnas.rename(columns={'PRIMER_APELLIDO_censo':'PRIMER_APELLIDO',
                                                          'SEGUNDO_APELLIDO_censo':'SEGUNDO_APELLIDO',
                                                          'PRIMER_NOMBRE_censo':'PRIMER_NOMBRE',
                                                          'SEGUNDO_NOMBRE_censo':'SEGUNDO_NOMBRE',
                                                          'EDAD_censo':'EDAD','SEXO_censo':'SEXO'})
        #concatenamos ambos dataframes
        Dato_finales = pd.concat([Datos_finales_usuarios,Datos_censo_final],  axis=0).drop_duplicates()

        #Buscamos Duplicados en el DataFrame y los eliminamos, tambien se depuran inconsistencias
        Data_final_usuarios = Dato_finales.sort_values(by=["IDENTIFICACION", "PRIMER_APELLIDO"],  ascending=False).drop_duplicates(subset=["IDENTIFICACION"])
        Data_final_usuarios = Data_final_usuarios.loc[~Data_final_usuarios["IDENTIFICACION"].isin([1])]

        #escribimos el excel con los datos del Dataframe final
        Data_final_usuarios.to_excel (r'C:\Users\Ricardo Roa\Desktop\Nuevo_Censo.xlsx', index = False, header=True)
        return Data_final_usuarios

        
# Fase 4 : Conteo de especialidades en el primer semestre 2019 con la clase Especialidades
class Especialidades(Datacollect):
    #declaramos las convencionas y llamamos a la clase Datacollect
    def __init__(self, especialidad=pd.DataFrame, Data_Res=pd.DataFrame, Conteo_I=pd.DataFrame, Conteo_II=pd.DataFrame, Recuento_I=int, Recuento_II=int):
        Datacollect.__init__(self, conteo = int)
        self.especialidad = especialidad
        self.Data_Res = Data_Res
        self.Conteo_I = Conteo_I
        self.Recuento_I = Recuento_I
        self.Conteo_II = Conteo_II
        self.Recuento_II = Recuento_II

    
    #creamos el grupo y subgrupo de los procedimientos quirurgicos en el dataframe de las consultas
    def crea_subgrupo_grupo(self):
        #Declaramos las convenciones
        especialidad = self.especialidad
        Data_Res = self.Data_Res
        
        try:
            #cargamos los archivos correspondientes a las consultas y resolucion 1896
            especialidad = pd.read_excel(r'D:\datos\x.xlsx')
            Data_Res = pd.read_excel(r'C:\Users\Ricardo Roa\Documents\rips2019\RES1896.xlsx')
        except:
            print("Error no se pudieron cargar los datos")

        
        #Convertimos la columna codigo de consulta en un String
        especialidad[["Codigo_consulta"]]=especialidad["Codigo_consulta"].astype(str)
        
        #realizamos un substring para crear las columnas grupo y subgrupo
        especialidad[["Subgrupo"]]=especialidad.Codigo_consulta.str[2:5]
        especialidad[["Grupo"]]=especialidad.Codigo_consulta.str[2:6]
        
        #realizamos un inner join con los dos Dataframes
        Data_Final_Especialidad_Res =pd.merge(left=especialidad, right=Data_Res, how="inner",  
                        left_on=['Subgrupo'], right_on=['Subgrupo'],
                        suffixes=('_data', '_res'),)

        self.especialidad = especialidad
        return Data_Final_Especialidad_Res.head()
    
    #creamos la columna especialidad en el archivo consultas
    def crea_especialidades(self):
        #creamos la convencion
        especialidad = self.especialidad
        #cargamos el archivo con el codigo de las consultas
        Datos_Consultas = pd.read_excel(r'C:\Users\Ricardo Roa\Documents\rips2019\CONSULTAAC.xlsx')
        #renombramos
        Datos_Consultas = Datos_Consultas.rename(columns={"Código de consulta":"Codigo_consulta"})
        #Realizamos un inner join con ambos DataFrames
        Nuevo_Datos_Consulta = pd.merge(especialidad, Datos_Consultas, how="inner",
                                         left_on=["Codigo_consulta"], right_on=["Codigo_consulta"],
                                         suffixes=('_data', '_con'),)
        
        self.especialidad = Nuevo_Datos_Consulta
        return Nuevo_Datos_Consulta.head()
    
    def conteo_especialidad(self):
        Conteo_I = self.Conteo_I
        Recuento_I = self.Recuento_I
        Conteo_II = self.Conteo_II
        Recuento_II = self.Recuento_II
        Nuevo_Datos_Consulta = self.especialidad
        #realizamos el conteo de consultas por especialidaddes realizadas en el primer semestre del 2019
        Datos_I_Semestre = Nuevo_Datos_Consulta.loc[
            (Nuevo_Datos_Consulta["Año"]==2019) & (Nuevo_Datos_Consulta["Mes"]>=1) & 
            (Nuevo_Datos_Consulta["Mes"]<7)]
        Conteo_I_Semestre = Datos_I_Semestre.value_counts("Especialidad")
        
         #realizamos el conteo de consultas por especialidaddes realizadas en el segundo semestre del 2019
        Datos_II_Semestre = Nuevo_Datos_Consulta.loc[
            (Nuevo_Datos_Consulta["Año"]==2019) & (Nuevo_Datos_Consulta["Mes"]>=7) & 
            (Nuevo_Datos_Consulta["Mes"]<=12)]
        Conteo_II_Semestre = Datos_II_Semestre.value_counts("Especialidad")
    
        #convertimos los resultados del primer semestre en un nuevo Dataframe
        ConteoFinal_I_Semestre = pd.DataFrame(Conteo_I_Semestre)
        ConteoFinal_I_Semestre = ConteoFinal_I_Semestre.rename(columns={0:"Totalizado"})
    
        #convertimos los resultados del segundo semestre en un nuevo Dataframe
        ConteoFinal_II_Semestre = pd.DataFrame(Conteo_II_Semestre)
        ConteoFinal_II_Semestre = ConteoFinal_II_Semestre.rename(columns={0:"Totalizado"})
        
        
        #Reiniciamos el index del dataframe el ordenamiento de las filas y columnas
        ConteoFinal_I_Semestre = ConteoFinal_I_Semestre.reset_index()
        ConteoFinal_II_Semestre = ConteoFinal_II_Semestre.reset_index()
        
        #se realiza conteo de la poblacion en el I semestre del 2019 y calculamos la intensidad de uso
        Recuento_I_Sem_P = Nuevo_Datos_Consulta.loc[(Nuevo_Datos_Consulta["Año"]==2019) & (Nuevo_Datos_Consulta["Mes"]>=1) &  (Nuevo_Datos_Consulta["Mes"]<7)
                                                   ].groupby('Iduser').Iduser.apply(lambda x: len(x.unique())).count()
        
        #se realiza conteo de la poblacion en el II semestre del 2019 y calculamos la intensidad de uso
        Recuento_II_Sem_P = Nuevo_Datos_Consulta.loc[(Nuevo_Datos_Consulta["Año"]==2019) & (Nuevo_Datos_Consulta["Mes"]>=7) &  (Nuevo_Datos_Consulta["Mes"]<=12)
                                                   ].groupby('Iduser').Iduser.apply(lambda x: len(x.unique())).count()

        
        lista_datos_I = []
        lista_datos_Fr_I = []
        lista_datos_II = []
        lista_datos_Fr_II = []
        contador = 0
        conteo = self.conteo
        
        for item in ConteoFinal_I_Semestre.Totalizado:
            #insertamos los registros
            total = item / Recuento_I_Sem_P
            ntotal = round(total,5)
            lista_datos_I.insert(contador,ntotal)
            #segunda insercion
            Fr = item / conteo
            Frtotal = round(Fr,5)
            lista_datos_Fr_I.insert(contador,Frtotal)
            contador = contador + 1
            #renombramos los dataframes y ejecutamos las inserciones
        valores_I = pd.DataFrame(lista_datos_I)
        valores_I = valores_I.rename(columns={0:"Valores"})
        ConteoFinal_I_Semestre[["Valores"]] = valores_I["Valores"]
        valores_fr_I = pd.DataFrame(lista_datos_Fr_I)
        valores_fr_I=  valores_fr_I.rename(columns={0:"Fr"})
        ConteoFinal_I_Semestre[["Fr"]] = valores_fr_I["Fr"]
        
        for item in ConteoFinal_II_Semestre.Totalizado:
            #insertamos los registros
            total = item / Recuento_II_Sem_P
            ntotal = round(total,5)
            lista_datos_II.insert(contador,ntotal)
            #segunda insercion
            Fr = item / conteo
            Frtotal = round(Fr,5)
            lista_datos_Fr_II.insert(contador,Frtotal)
            contador = contador + 1
            #renombramos los dataframes y ejecutamos las inserciones
        valores_II = pd.DataFrame(lista_datos_II)
        valores_II = valores_II.rename(columns={0:"Valores"})
        ConteoFinal_II_Semestre[["Valores"]] = valores_II["Valores"]
        valores_fr_II = pd.DataFrame(lista_datos_Fr_II)
        valores_fr_II=  valores_fr_II.rename(columns={0:"Fr"})
        ConteoFinal_II_Semestre[["Fr"]] = valores_fr_II["Fr"]
            
        #declaramos las convensiones y le asignamos los dataframes para el calculo de la intensidad, frecuencia y extension de uso
        self.Conteo_I = ConteoFinal_I_Semestre
        self.Conteo_II = ConteoFinal_II_Semestre
        self.conteo = conteo
        self.Recuento_I = Recuento_I_Sem_P
        self.Recuento_II = Recuento_II_Sem_P
    
    def Visualiza_I_Semestre(self):
        ConteoFinal_I_Semestre =  self.Conteo_I
        return ConteoFinal_I_Semestre
    
    def Visualiza_II_Semestre(self):
        ConteoFinal_II_Semestre =  self.Conteo_II
        return ConteoFinal_II_Semestre
    
        
    def calculo_int_fr_ext(self):
        ConteoFinal_I_Semestre = self.Conteo_I
        ConteoFinal_II_Semestre = self.Conteo_II
        conteo = self.conteo
        Recuento_I = self.Recuento_I
        Recuento_II = self.Recuento_II
        Frecuencia_Uso_I = round(ConteoFinal_I_Semestre.Fr.sum()*100)
        Intensidad_Uso_I = round(ConteoFinal_I_Semestre.Valores.sum()*100)
        Extension_Uso_I= round(( Recuento_I / conteo ) * 100)
        Frecuencia_Uso_II = round(ConteoFinal_II_Semestre.Fr.sum()*100)
        Intensidad_Uso_II = round(ConteoFinal_II_Semestre.Valores.sum()*100)
        Extension_Uso_II= round(( Recuento_II / conteo ) * 100)
        Datos = {'I_2019': pd.Series([Extension_Uso_I,Frecuencia_Uso_I,Intensidad_Uso_I], index=['Extension de uso consulta','Frecuencia de uso consulta','Intensidad de uso consulta']),
                 'II_2019': pd.Series([Extension_Uso_II,Frecuencia_Uso_II,Intensidad_Uso_II], index=['Extension de uso consulta','Frecuencia de uso consulta','Intensidad de uso consulta'])
                }
        Datos_Estadisticos = pd.DataFrame(Datos)
        return Datos_Estadisticos
    
   
        
    


In [7]:
limpiar = Concatenacion()
limpiar.lee_dataframe()
limpiar.join()
limpiar.elimina_duplicados_nulos()
limpiar.limpiardatos()
limpiar.concatena()

,IDENTIFICACION,PRIMER_APELLIDO,SEGUNDO_APELLIDO,PRIMER_NOMBRE,SEGUNDO_NOMBRE,EDAD,SEXO,TIPO_DOCUMENTO
59945,YB1023370,BAZZONI,NaN,VITTORIA,NaN,30,F,PA
698251,YA9388862,AMARU,NaN,MARCO,NaN,57,M,DE
494380,YA8371228,DI,SOMMA,ANNUNZIATA,NaN,47,F,PA
715295,YA4651723,FERLITO,NaN,RAIMONDO,NaN,44,M,DE
725012,YA4506066,BIANCOLI,NaN,CLAUDIO,NaN,30,M,DE
...,...,...,...,...,...,...,...,...
471644,179728,DUARTE,GUERRERO,FERNANDO,NaN,47,M,CC
208426,179715,GUZMAN,QUINTERO,WILSON,NaN,48,M,CC
593237,178584,MORENO,NaN,ANDRES,NaN,73,M,CC
534103,177235,GARCIA,NaN,JORGE,ISAAC,51,M,CC


In [8]:
visualiza_I = Especialidades()
visualiza_I.crea_subgrupo_grupo()
visualiza_I.crea_especialidades()
visualiza_I.conteo_especialidad()
visualiza_I.Visualiza_I_Semestre()

,Especialidad,Totalizado,Valores,Fr
0,Medicina General,162157,1.55809,1.32250
1,Otros,110699,1.06366,0.90283
2,Especialidad,29930,0.28758,0.24410
3,Optometria,4816,0.04627,0.03928
4,Cirugia General,2373,0.02280,0.01935
5,Ortopedia,2018,0.01939,0.01646
6,Medicina Interna,1960,0.01883,0.01599
7,Oftalmologia,1890,0.01816,0.01541
8,Urologia,1655,0.01590,0.01350
9,Psiquiatria,1388,0.01334,0.01132


In [9]:
visualiza_II = Especialidades()
visualiza_II.crea_subgrupo_grupo()
visualiza_II.crea_especialidades()
visualiza_II.conteo_especialidad()
visualiza_II.Visualiza_II_Semestre()

,Especialidad,Totalizado,Valores,Fr
0,Medicina General,223893,1.81300,1.82600
1,Otros,153933,1.24649,1.25543
2,Especialidad,21585,0.17479,0.17604
3,Optometria,5602,0.04536,0.04569
4,Cirugia General,1744,0.01412,0.01422
5,Ortopedia,1650,0.01336,0.01346
6,Oftalmologia,1464,0.01185,0.01194
7,Medicina Interna,1411,0.01143,0.01151
8,Urologia,1352,0.01095,0.01103
9,Otorrino,661,0.00535,0.00539


In [10]:
calculo = Especialidades()
calculo.crea_subgrupo_grupo()
calculo.crea_especialidades()
calculo.conteo_especialidad()
calculo.calculo_int_fr_ext()

,I_2019,II_2019
Extension de uso consulta,85,101
Frecuencia de uso consulta,261,338
Intensidad de uso consulta,308,335
